In [9]:
import copy
import numpy as np
from scipy.stats import ks_2samp
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from Team_Win_Analysis_Runner_withcaptain_vice import *
from pathlib import Path
from scipy.stats import ks_2samp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde
import numpy as np
from pathlib import Path

# Assume `data` is already loaded and `Error_Bound`, `Tau`, `input_dir`, and `N_experts` are defined.

# Soft pastel color palette
color_palette = [
    '#A6CEE3',  # pastel blue
    '#FDBF6F',  # pastel orange
    '#B2DF8A',  # pastel green
    '#FB9A99',  # pastel red
    '#CAB2D6',  # pastel purple
    '#FFFF99',  # pastel yellow
    '#1F78B4',  # soft blue
    '#33A02C',  # soft green
    '#FB8072',  # coral
    '#80B1D3'   # light steel blue
]
N_iterations=10000
for N_experts in range(3,6,1):
    Base_folder=f"Base_{N_experts}"
    for correlation in [0.33,0.5,0.67]:
        Correlation_folder=f"{Base_folder}/{correlation}"
        for (Tau,Error_Bound) in [(0.1,0.1),(0.2,0.1),(0.3,0.1),(0.4,0.1),(0.2,0.02),(0.2,0.06),(0.2,0.14)]:
            input_dir=f"{Correlation_folder}/Tau={Tau}_ErrorBound={Error_Bound}"
            data=pd.read_excel(f"{input_dir}/Simulation Output.xlsx", sheet_name=None)
            Export_summary = {}
            Log_odds = {}
            for Config in list(data.keys())[:-1]:
                K = data[Config]
                Stat = pd.DataFrame()
                log = pd.DataFrame()
                Index=[(i,1,2) for i in range(1,N_experts+1)]
                # Winnings stats
                for i in Index:
                    l = K[f"Winnings_{i}"][K[f"Winnings_{i}"] != 0].describe().round(3)
                    l.loc["n_zeros"] = N_iterations - l["count"]
                    l.loc["Mean (including zeros)"] = K[f"Winnings_{i}"].mean()
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
                    Stat[f"Expert {i} Winnings (Non-zero)"] = l

                # Total selections
                

                # Analytical
                for i in Index:
                    l = K[f"Analytical_{i}"].describe().round(3)
                    l.loc["n_zeros"] = "NA"
                    l.loc["Mean (including zeros)"] = "NA"
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
                    Stat[f"Analytical_{i}"] = l

                # Random
                for i in Index:
                    l = K[f"Random_{i}"].describe().round(3)
                    l.loc["n_zeros"] = "NA"
                    l.loc["Mean (including zeros)"] = "NA"
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
                    Stat[f"Random_{i}"] = l

                # Log-odds (really, log of ratio between means)
                for i in Index:
                    ratio = (Stat.loc["mean", f"Analytical_{i}"] / Stat.loc["mean", f"Random_{i}"])*(1-Tau)/Tau
                    #log_ratio = np.log(ratio)
                    log[f"Expert {i}"] = pd.Series(ratio)
                for i in Index:
                    l = K[f"Total_{i}"].describe().round(3)
                    l.loc["n_zeros"] = "NA"
                    l.loc["Mean (including zeros)"] = "NA"
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"] = float(log[f"Expert {i}"].iloc[0])
                    Stat[f"Total_Sel_{i}"] = l

                ordered_cols = []

            # Group all winnings together
                for i in Index:
                    ordered_cols.append(f"Expert {i} Winnings (Non-zero)")

                # Then all totals
                for i in Index:
                    ordered_cols.append(f"Total_Sel_{i}")

                # Then all analytical
                for i in Index:
                    ordered_cols.append(f"Analytical_{i}")

                # Then all random
                for i in Index:
                    ordered_cols.append(f"Random_{i}")

                # Apply the reordering
                Stat = Stat[ordered_cols]
                Export_summary[Config] = Stat
            with pd.ExcelWriter(f"{input_dir}/Summary.xlsx", engine="openpyxl") as writer:
                for sheet_name, df in Export_summary.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=True)
            data=pd.read_excel(f"{input_dir}/Summary.xlsx",sheet_name=None)
            for Team_set_type in data.keys():
                directory = Path(f"{input_dir}/Plots")
                directory.mkdir(parents=True, exist_ok=True)

                fig = make_subplots(
                    rows=3, cols=2,
                    subplot_titles=(
                        "Winnings (Zeros not included)",
                        "Winnings (Zeros included)",
                        "Total Selections",
                        "Analytical Selections",
                        "Random Selections",
                        ""
                    ),
                    horizontal_spacing=0.15,
                    vertical_spacing=0.15
                )

                fig.update_layout(
                    title_text=f"Expert Team Distributions for {Team_set_type} (Error Bound: {Error_Bound}, Tau: {Tau})",
                    height=900, width=1200,
                    showlegend=True,
                    legend=dict(title="Experts", traceorder="normal")
                )

                def add_kde_trace(series, name, row, col, color, showlegend):
                    if len(series) > 1:
                        kde = gaussian_kde(series)
                        x_vals = np.linspace(min(series), max(series), 200)
                        y_vals = kde(x_vals)
                        fig.add_trace(
                            go.Scatter(x=x_vals, y=y_vals, mode='lines', name=name, line=dict(color=color), legendgroup=name, showlegend=showlegend),
                            row=row, col=col
                        )

                for i in Index:
                    color = color_palette[(i[0] - 1) % len(color_palette)]
                    legend_name = f"Expert {i[0]}"

                    series = data[Team_set_type][f"Expert {i} Winnings (Non-zero)"]
                    series_no_zero = series[series > 0]
                    add_kde_trace(series_no_zero, legend_name, 1, 1, color, showlegend=True)
                    add_kde_trace(series, legend_name, 1, 2, color, showlegend=False)

                    series_total = data[Team_set_type][f"Total_{i}"]
                    add_kde_trace(series_total, legend_name, 2, 1, color, showlegend=False)

                    series_analytical = data[Team_set_type][f"Analytical_{i}"]
                    add_kde_trace(series_analytical, legend_name, 2, 2, color, showlegend=False)

                    series_random = data[Team_set_type][f"Random_{i}"]
                    add_kde_trace(series_random, legend_name, 3, 1, color, showlegend=False)

                fig.update_xaxes(title_text="Winnings", row=1, col=1)
                fig.update_xaxes(title_text="Winnings", row=1, col=2)
                fig.update_xaxes(title_text="Selections", row=2, col=1)
                fig.update_xaxes(title_text="Selections", row=2, col=2)
                fig.update_xaxes(title_text="Selections", row=3, col=1)

                fig.update_yaxes(title_text="Density", row=1, col=1)
                fig.update_yaxes(title_text="Density", row=1, col=2)
                fig.update_yaxes(title_text="Density", row=2, col=1)
                fig.update_yaxes(title_text="Density", row=2, col=2)
                fig.update_yaxes(title_text="Density", row=3, col=1)

                fig.write_html(f"{directory}/Summary_{Team_set_type}.html")
            P_value={}
            for Team_set_type in data.keys():
                Stat={}
                directory = input_dir
                for i in Index:
                    # Extract the winnings data for the two groups
                    Stat[f"Expert_{i}"] = {}
                    for j in ["Total", "Analytical", "Random"]:
                        data1 = data[Team_set_type][f"{j}_{i}"][data[Team_set_type][f"Winnings_{i}"] > 0]
                        data2 = data[Team_set_type][f"{j}_{i}"][data[Team_set_type][f"Winnings_{i}"] == 0]
                        
                        # Perform the two-sample KS test
                        statistic, p_value = ks_2samp(data1, data2)
                        Stat[f"Expert_{i}"][j] = p_value
                P_value[Team_set_type] = pd.DataFrame(Stat)
                with pd.ExcelWriter(f"{directory}/P_value.xlsx", engine="openpyxl") as writer:
                    for sheet_name, df in P_value.items():
                        df.to_excel(writer, sheet_name=sheet_name, index=True)

ValueError: array must not contain infs or NaNs

In [ ]:
import copy
import numpy as np
from scipy.stats import ks_2samp
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from Team_Win_Analysis_Runner_withcaptain_vice import *
from pathlib import Path
from scipy.stats import ks_2samp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde
import numpy as np
from pathlib import Path

# Assume `data` is already loaded and `Error_Bound`, `Tau`, `input_dir`, and `N_experts` are defined.

# Soft pastel color palette
color_palette = [
    "#66c2a5",  # soft green
    "#fc8d62",  # soft orange
    "#8da0cb",  # soft blue
    "#e78ac3",  # soft pink
    "#a6d854",  # soft lime
    "#ffd92f",  # soft yellow
    "#e5c494",  # beige/brown
    "#b3b3b3",  # soft grey
    "#a1c9f4",  # light blue
    "#fdbf6f"   # pastel orange
]
color_palette = [
    "#1f77b4",  # rich blue
    "#ff7f0e",  # strong orange
    "#2ca02c",  # bold green
    "#d62728",  # crimson red
    "#9467bd",  # medium purple
    "#8c564b",  # soft brown
    "#e377c2",  # vibrant pink
    "#7f7f7f",  # mid grey
    "#bcbd22",  # olive
    "#17becf"   # teal/cyan
]
color_palette = [
    "#4C78A8",  # deep steel blue
    "#F58518",  # warm amber
    "#54A24B",  # rich green
    "#E45756",  # coral red
    "#B279A2",  # plum violet
    "#FF9DA6",  # light pink rose
    "#9D755D",  # soft bronze brown
    "#BAB0AC",  # soft muted grey
    "#72B7B2",  # turquoise
    "#F2CF5B"   # golden yellow
]
color_palette = [
    "#386cb0",  # elegant royal blue
    "#fdb462",  # tangerine gold
    "#7fc97f",  # fresh grass green
    "#ef3b2c",  # true vivid red
    "#984ea3",  # deep orchid purple
    "#a6cee3",  # powder sky blue
    "#f58e8c",  # softened salmon pink
    "#e31a1c",  # strong cardinal red
    "#33a02c",  # vibrant forest green
    "#ff7f00"   # bright amber orange
]
N_iterations=10000
for N_experts in range(3,6,1):
    Base_folder=f"Base_{N_experts}"
    for correlation in [0.33,0.5,0.67]:
        Correlation_folder=f"{Base_folder}/{correlation}"
        for (Tau,Error_Bound) in [(0.1,0.1),(0.2,0.1),(0.3,0.1),(0.4,0.1),(0.2,0.02),(0.2,0.06),(0.2,0.14)]:
            input_dir=f"{Correlation_folder}/Tau={Tau}_ErrorBound={Error_Bound}"
            data=pd.read_excel(f"{input_dir}/Simulation Output.xlsx", sheet_name=None)
            Export_summary = {}
            Log_odds = {}
            for Config in list(data.keys())[:-1]:
                K = data[Config]
                Stat = pd.DataFrame()
                log = pd.DataFrame()
                Index=[(i,1,2) for i in range(1,N_experts+1)]
                # Winnings stats
                for i in Index:
                    l = K[f"Winnings_{i}"][K[f"Winnings_{i}"] != 0].describe().round(3)
                    l.loc["n_zeros"] = N_iterations - l["count"]
                    l.loc["Mean (including zeros)"] = K[f"Winnings_{i}"].mean()
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
                    Stat[f"Expert {i} Winnings (Non-zero)"] = l

                # Total selections
                

                # Analytical
                for i in Index:
                    l = K[f"Analytical_{i}"].describe().round(3)
                    l.loc["n_zeros"] = "NA"
                    l.loc["Mean (including zeros)"] = "NA"
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
                    Stat[f"Analytical_{i}"] = l

                # Random
                for i in Index:
                    l = K[f"Random_{i}"].describe().round(3)
                    l.loc["n_zeros"] = "NA"
                    l.loc["Mean (including zeros)"] = "NA"
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"]="NA"
                    Stat[f"Random_{i}"] = l

                # Log-odds (really, log of ratio between means)
                for i in Index:
                    ratio = (Stat.loc["mean", f"Analytical_{i}"] / Stat.loc["mean", f"Random_{i}"])*(1-Tau)/Tau
                    #log_ratio = np.log(ratio)
                    log[f"Expert {i}"] = pd.Series(ratio)
                for i in Index:
                    l = K[f"Total_{i}"].describe().round(3)
                    l.loc["n_zeros"] = "NA"
                    l.loc["Mean (including zeros)"] = "NA"
                    l.loc["Selection Ratio (Weighted) (Analytical/Random)"] = float(log[f"Expert {i}"].iloc[0])
                    Stat[f"Total_Sel_{i}"] = l

                ordered_cols = []

            # Group all winnings together
                for i in Index:
                    ordered_cols.append(f"Expert {i} Winnings (Non-zero)")

                # Then all totals
                for i in Index:
                    ordered_cols.append(f"Total_Sel_{i}")

                # Then all analytical
                for i in Index:
                    ordered_cols.append(f"Analytical_{i}")

                # Then all random
                for i in Index:
                    ordered_cols.append(f"Random_{i}")

                # Apply the reordering
                Stat = Stat[ordered_cols]
                Export_summary[Config] = Stat
            with pd.ExcelWriter(f"{input_dir}/Summary.xlsx", engine="openpyxl") as writer:
                for sheet_name, df in Export_summary.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=True)
            import shutil

            shutil.rmtree(f"{input_dir}/Plots")
            directory = Path(f"{input_dir}/Plots")
            directory.mkdir(parents=True, exist_ok=True)
            for Team_set_type in list(data.keys())[:-1]:
                
                

                fig = make_subplots(
                    rows=3, cols=2,
                    subplot_titles=(
                        "Winnings (Zeros not included)",
                        "Winnings (Zeros included)",
                        "Total Selections",
                        "Analytical Selections",
                        "Random Selections",
                        ""
                    ),
                    horizontal_spacing=0.15,
                    vertical_spacing=0.15
                )

                fig.update_layout(
                    title_text=f"Expert Team Distributions for {Team_set_type} (Error Bound: {Error_Bound}, Tau: {Tau}, Correlation: {correlation}, Experts: {N_experts})",
                    height=900, width=1200,
                    showlegend=True,
                    legend=dict(title="Experts", traceorder="normal")
                )

                def add_kde_trace(series, name, row, col, color, showlegend):
                    if len(series) > 1:
                        kde = gaussian_kde(series)
                        x_vals = np.linspace(min(series), max(series), 200)
                        y_vals = kde(x_vals)
                        fig.add_trace(
                            go.Scatter(x=x_vals, y=y_vals, mode='lines', name=name, line=dict(color=color), legendgroup=name, showlegend=showlegend),
                            row=row, col=col
                        )

                for i in Index:
                    color = color_palette[(i[0] - 1) % len(color_palette)]
                    legend_name = f"Expert {i[0]}"

                    series = data[Team_set_type][f"Winnings_{i}"]
                    series_no_zero = series[series > 0]
                    add_kde_trace(series_no_zero, legend_name, 1, 1, color, showlegend=True)
                    add_kde_trace(series, legend_name, 1, 2, color, showlegend=False)

                    series_total = data[Team_set_type][f"Total_{i}"]
                    add_kde_trace(series_total, legend_name, 2, 1, color, showlegend=False)

                    series_analytical = data[Team_set_type][f"Analytical_{i}"]
                    add_kde_trace(series_analytical, legend_name, 2, 2, color, showlegend=False)

                    series_random = data[Team_set_type][f"Random_{i}"]
                    add_kde_trace(series_random, legend_name, 3, 1, color, showlegend=False)

                fig.update_xaxes(title_text="Winnings", row=1, col=1)
                fig.update_xaxes(title_text="Winnings", row=1, col=2)
                fig.update_xaxes(title_text="Selections", row=2, col=1)
                fig.update_xaxes(title_text="Selections", row=2, col=2)
                fig.update_xaxes(title_text="Selections", row=3, col=1)

                fig.update_yaxes(title_text="Density", row=1, col=1)
                fig.update_yaxes(title_text="Density", row=1, col=2)
                fig.update_yaxes(title_text="Density", row=2, col=1)
                fig.update_yaxes(title_text="Density", row=2, col=2)
                fig.update_yaxes(title_text="Density", row=3, col=1)
            
                directory_html = Path(f"{directory}/HTML")
                directory_html.mkdir(parents=True, exist_ok=True)

                fig.write_html(f"{directory_html}/Summary_{Team_set_type}.html")
                directory_png=Path(f"{directory}/PNG")
                directory_png.mkdir(parents=True, exist_ok=True)
                fig.write_image(f"{directory_png}/Summary_{Team_set_type}.png",engine="kaleido")
            P_value={}
            for Team_set_type in list(data.keys())[:-1]:
                Stat={}
                directory = input_dir
                for i in Index:
                    # Extract the winnings data for the two groups
                    Stat[f"Expert_{i}"] = {}
                    for j in ["Total", "Analytical", "Random"]:
                        data1 = data[Team_set_type][f"{j}_{i}"][data[Team_set_type][f"Winnings_{i}"] > 0]
                        data2 = data[Team_set_type][f"{j}_{i}"][data[Team_set_type][f"Winnings_{i}"] == 0]
                        
                        # Perform the two-sample KS test
                        statistic, p_value = ks_2samp(data1, data2)
                        Stat[f"Expert_{i}"][j] = p_value
                P_value[Team_set_type] = pd.DataFrame(Stat)
                with pd.ExcelWriter(f"{directory}/P_value.xlsx", engine="openpyxl") as writer:
                    for sheet_name, df in P_value.items():
                        df.to_excel(writer, sheet_name=sheet_name, index=True)

KeyboardInterrupt: 

In [17]:

data[Team_set_type]

,"Winnings_(1, 1, 2)","Analytical_(1, 1, 2)","Random_(1, 1, 2)","Total_(1, 1, 2)","Winnings_(2, 1, 2)","Analytical_(2, 1, 2)","Random_(2, 1, 2)","Total_(2, 1, 2)","Winnings_(3, 1, 2)","Analytical_(3, 1, 2)","Random_(3, 1, 2)","Total_(3, 1, 2)","Winnings_(4, 1, 2)","Analytical_(4, 1, 2)","Random_(4, 1, 2)","Total_(4, 1, 2)","Winnings_(5, 1, 2)","Analytical_(5, 1, 2)","Random_(5, 1, 2)","Total_(5, 1, 2)"
0,0.0,3,163,166,0.0,2,157,159,0.000000,20,164,184,0.000000,71,149,220,73.800738,104,167,271
1,0.0,0,173,173,0.0,3,160,163,0.000000,25,144,169,0.000000,63,144,207,69.444444,109,179,288
2,0.0,1,161,162,0.0,1,139,140,0.000000,23,145,168,0.000000,65,176,241,69.204152,110,179,289
3,0.0,0,180,180,0.0,5,134,139,0.000000,20,142,162,81.300813,60,186,246,0.000000,115,158,273
4,0.0,3,152,155,0.0,4,158,162,0.000000,23,161,184,84.033613,62,176,238,0.000000,108,153,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0,143,143,0.0,2,176,178,118.343195,25,144,169,0.000000,50,178,228,0.000000,123,159,282
9996,0.0,1,159,160,0.0,7,178,185,0.000000,29,150,179,92.592593,58,158,216,0.000000,105,155,260
9997,0.0,1,173,174,0.0,5,149,154,0.000000,18,178,196,0.000000,57,150,207,74.349442,119,150,269
9998,0.0,1,144,145,0.0,1,139,140,0.000000,33,170,203,90.090090,50,172,222,0.000000,115,175,290
